In [36]:
import pandas as pd
import csv
import pyproj
import math
import numpy as np
from math import sin, cos, sqrt, atan2, radians

# Read in GPS log data from Arduino and preprocess it
gps_log_df = pd.read_csv("../Trial3/Arduino/GPSLOG1.csv").drop_duplicates()
gps_log_df["TimeStamps"] = pd.to_datetime(pd.to_datetime(gps_log_df["Date"] + " " + gps_log_df["Time"]))
gps_log_df = pd.concat([gps_log_df["TimeStamps"], gps_log_df[["GPS", "Latitude", "Longitude"]]], axis=1)

# Group the data by GPS sensor and sort by timestamps
gps_log_df_1 = gps_log_df[gps_log_df["GPS"].values == "GPS1"].reset_index().sort_values(by="TimeStamps")
gps_log_df_2 = gps_log_df[gps_log_df["GPS"].values == "GPS2"].reset_index().sort_values(by="TimeStamps")
gps_log_df_3 = gps_log_df[gps_log_df["GPS"].values == "GPS3"].reset_index().sort_values(by="TimeStamps")

# Merge the three dataframes into one
merged_gps_log_df = pd.merge(gps_log_df_1, gps_log_df_2, on='TimeStamps', how='outer')
merged_gps_log_df = pd.merge(merged_gps_log_df, gps_log_df_3, on='TimeStamps', how='outer')

# Rename columns for clarity
merged_gps_log_df.rename(columns={'GPS_x': 'GPS1', 'Latitude_x': 'Latitude1', 'Longitude_x': 'Longitude1',
                                  'GPS_y': 'GPS2', 'Latitude_y': 'Latitude2', 'Longitude_y': 'Longitude2',
                                  'GPS': 'GPS3', 'Latitude': 'Latitude3', 'Longitude': 'Longitude3'},
                         inplace=True)

# Convert GPS coordinates to UTM coordinates
def convert_to_utm(df, lat_col, long_col):
    utm_proj = pyproj.Proj(proj='utm', zone=32, ellps='WGS84') # set UTM projection with zone number and ellipsoid
    utm_coords = []
    for lat, lon in zip(df[lat_col], df[long_col]):
        easting, northing = utm_proj(lon, lat) # convert latitude and longitude to UTM easting and northing
        utm_coords.append((easting, northing))
    df[lat_col + '_utm_easting'], df[long_col + '_utm_northing'] = zip(*utm_coords) # add new columns to the DataFrame
    return df

merged_gps_log_df = convert_to_utm(merged_gps_log_df, 'Latitude1', 'Longitude1')
merged_gps_log_df = convert_to_utm(merged_gps_log_df, 'Latitude2', 'Longitude2')
merged_gps_log_df = convert_to_utm(merged_gps_log_df, 'Latitude3', 'Longitude3')

only_utm_merged_df = merged_gps_log_df.loc[:,["TimeStamps", "Latitude1_utm_easting", "Longitude1_utm_northing",
                                              "Latitude2_utm_easting","Longitude2_utm_northing", 
                                              "Latitude3_utm_easting","Longitude3_utm_northing"]].drop_duplicates().reset_index()

# Calculate angle between subsequent coordinates
def angle_between_coordinates(lat1, lon1, lat2, lon2, lat3, lon3, lat4, lon4):
    vector1 = np.array([lat1 - lat2, lon1 - lon2])
    vector2 = np.array([lat3 - lat4, lon3 - lon4])
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    if magnitude1 == 0 or magnitude2 == 0:
        return np.nan

    dot_product = np.dot(vector1,vector2)
    cos_angle = dot_product / (magnitude1 * magnitude2)
    angle = np.arccos(cos_angle)
    return np.degrees(angle)


In [100]:
#Orientation Change Calculation
trueFalse = []
orientation = []
for ts in range(len(only_utm_merged_df)):
    
    init_points = []
    final_points = []
    if ts>0 and ts<=len(only_utm_merged_df)-1:
        #only_utm_merged_df.loc[ts,"orientation"] = 0
        if only_utm_merged_df.loc[ts-1,"Latitude1_utm_easting"] != np.inf:
            init_points.append("Latitude1_utm_easting")
            init_points.append("Longitude1_utm_northing")
        if only_utm_merged_df.loc[ts-1,"Latitude2_utm_easting"] != np.inf:
            init_points.append("Latitude2_utm_easting")
            init_points.append("Longitude2_utm_northing")
        if only_utm_merged_df.loc[ts-1,"Latitude3_utm_easting"] != np.inf:
            init_points.append("Latitude3_utm_easting")
            init_points.append("Longitude3_utm_northing")
        if only_utm_merged_df.loc[ts,"Latitude1_utm_easting"] != np.inf:
            final_points.append("Latitude1_utm_easting")
            final_points.append("Longitude1_utm_northing")
        if only_utm_merged_df.loc[ts,"Latitude2_utm_easting"] != np.inf:
            final_points.append("Latitude2_utm_easting")
            final_points.append("Longitude2_utm_northing")
        if only_utm_merged_df.loc[ts,"Latitude3_utm_easting"] != np.inf:
            final_points.append("Latitude3_utm_easting")
            final_points.append("Longitude3_utm_northing")
    
    #print(init_points)
    #print(final_points)
    
    if len(init_points) < 4 and len(final_points) < 4:
        orientation.append(0)
    
    if init_points == final_points and (len(init_points) >= 4 and len(final_points) >= 4):
        trueFalse.append(True)
        angle = angle_between_coordinates(only_utm_merged_df.loc[ts-1,init_points[0]],
                                          only_utm_merged_df.loc[ts-1,init_points[1]],
                                          only_utm_merged_df.loc[ts-1,init_points[2]],
                                          only_utm_merged_df.loc[ts-1,init_points[3]],
                                          only_utm_merged_df.loc[ts,final_points[0]],
                                          only_utm_merged_df.loc[ts,final_points[1]],
                                          only_utm_merged_df.loc[ts,final_points[2]],
                                          only_utm_merged_df.loc[ts,final_points[3]])
        orientation.append(angle)
    else:
        print(ts)
        orientation.append(np.nan)
        print(init_points)
        print(final_points)
        trueFalse.append(False)
    
    
off = pd.DataFrame({'trueFalse': trueFalse, 'orientation': orientation})

off


0
[]
[]
4
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude3_utm_easting', 'Longitude3_utm_northing']
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude2_utm_easting', 'Longitude2_utm_northing', 'Latitude3_utm_easting', 'Longitude3_utm_northing']
14
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude2_utm_easting', 'Longitude2_utm_northing', 'Latitude3_utm_easting', 'Longitude3_utm_northing']
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude2_utm_easting', 'Longitude2_utm_northing']
16
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude2_utm_easting', 'Longitude2_utm_northing']
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude2_utm_easting', 'Longitude2_utm_northing', 'Latitude3_utm_easting', 'Longitude3_utm_northing']
61
['Latitude1_utm_easting', 'Longitude1_utm_northing', 'Latitude2_utm_easting', 'Longitude2_utm_northing', 'Latitude3_utm_easting', 'Longitude3_utm_northing']
['Latitude1_utm_easting', 'Longitude1_

In [109]:
same_init_final_points = [None] * len(only_utm_merged_df)
orientation = [None] * len(only_utm_merged_df)

for ts in range(len(only_utm_merged_df)):
    
    init_points = []
    final_points = []
    
    if ts > 0 and ts <= len(only_utm_merged_df)-1:
        if np.isfinite(only_utm_merged_df.iloc[ts-1]["Latitude1_utm_easting"]):
            init_points.extend(["Latitude1_utm_easting", "Longitude1_utm_northing"])
        if np.isfinite(only_utm_merged_df.iloc[ts-1]["Latitude2_utm_easting"]):
            init_points.extend(["Latitude2_utm_easting", "Longitude2_utm_northing"])
        if np.isfinite(only_utm_merged_df.iloc[ts-1]["Latitude3_utm_easting"]):
            init_points.extend(["Latitude3_utm_easting", "Longitude3_utm_northing"])
        if np.isfinite(only_utm_merged_df.iloc[ts]["Latitude1_utm_easting"]):
            final_points.extend(["Latitude1_utm_easting", "Longitude1_utm_northing"])
        if np.isfinite(only_utm_merged_df.iloc[ts]["Latitude2_utm_easting"]):
            final_points.extend(["Latitude2_utm_easting", "Longitude2_utm_northing"])
        if np.isfinite(only_utm_merged_df.iloc[ts]["Latitude3_utm_easting"]):
            final_points.extend(["Latitude3_utm_easting", "Longitude3_utm_northing"])
    
    if len(init_points) < 4 and len(final_points) < 4:
        orientation[ts] = 0
    
    if init_points == final_points and len(init_points) >= 4 and len(final_points) >= 4:
        same_init_final_points[ts] = True
        angle = angle_between_coordinates(
            only_utm_merged_df.iloc[ts-1][init_points[0]],
            only_utm_merged_df.iloc[ts-1][init_points[1]],
            only_utm_merged_df.loc[ts-1,init_points[2]],
            only_utm_merged_df.loc[ts-1,init_points[3]],
            only_utm_merged_df.loc[ts,final_points[0]],
            only_utm_merged_df.loc[ts,final_points[1]],
            only_utm_merged_df.loc[ts,final_points[2]],
            only_utm_merged_df.loc[ts,final_points[3]])
        orientation[ts] = angle
        
    else:
        orientation[ts] = np.nan
        same_init_final_points[ts] = False
    
    
off = pd.DataFrame({"TS":only_utm_merged_df.loc[:,"TimeStamps"],'same_init_final_points': same_init_final_points, 'orientation': orientation})

off.head(30)
            


,TS,same_init_final_points,orientation
0,2023-04-14 10:48:48,False,NaN
1,2023-04-14 10:48:49,True,0.000000
2,2023-04-14 10:48:50,True,3.890617
3,2023-04-14 10:48:51,True,3.890615
4,2023-04-14 10:48:52,False,NaN
5,2023-04-14 10:48:53,True,0.061535
6,2023-04-14 10:48:54,True,3.237654
7,2023-04-14 10:48:55,True,0.000000
8,2023-04-14 10:48:56,True,3.012856
9,2023-04-14 10:48:57,True,2.700334


In [93]:
off = pd.DataFrame({'trueFalse': trueFalse, 'orientation': orientation})

off

ValueError: All arrays must be of the same length

In [61]:
only_utm_merged_df.loc[0,"Latitude1_utm_easting"] != np.inf

True

In [39]:
range(len(only_utm_merged_df))

range(0, 119)

In [66]:
only_utm_merged_df

,index,TimeStamps,Latitude1_utm_easting,Longitude1_utm_northing,Latitude2_utm_easting,Longitude2_utm_northing,Latitude3_utm_easting,Longitude3_utm_northing,"(0, orientation)",orientation
0,0,2023-04-14 10:48:48,6.818069e+05,5.406171e+06,inf,inf,6.818005e+05,5.406171e+06,0,0.0
1,1,2023-04-14 10:48:49,6.818070e+05,5.406171e+06,inf,inf,6.818005e+05,5.406171e+06,0,NaN
2,2,2023-04-14 10:48:50,6.818071e+05,5.406171e+06,inf,inf,6.818006e+05,5.406170e+06,0,NaN
3,3,2023-04-14 10:48:51,6.818075e+05,5.406171e+06,inf,inf,6.818006e+05,5.406170e+06,0,NaN
4,4,2023-04-14 10:48:52,6.818077e+05,5.406171e+06,6.818000e+05,5.406171e+06,6.818006e+05,5.406170e+06,0,NaN
...,...,...,...,...,...,...,...,...,...,...
114,194,2023-04-14 10:50:41,inf,inf,6.818753e+05,5.406187e+06,inf,inf,0,NaN
115,195,2023-04-14 10:50:42,inf,inf,6.818761e+05,5.406186e+06,inf,inf,0,NaN
116,196,2023-04-14 10:50:43,inf,inf,6.818765e+05,5.406186e+06,inf,inf,0,NaN
117,197,2023-04-14 10:50:44,inf,inf,6.818768e+05,5.406186e+06,inf,inf,0,NaN


In [43]:
len(only_utm_merged_df)

119

In [51]:
only_utm_merged_df.loc[1,"Latitude1_utm_easting"] != np.inf

True

In [49]:
only_utm_merged_df.loc[1,"Latitude2_utm_easting"]

inf

In [71]:
only_utm_merged_df[1-1,init_points[1]]

IndexError: list index out of range

In [ ]:
'''angle = angle_between_coordinates(only_utm_merged_df.loc[98,"Latitude3_utm_easting"],
                                 only_utm_merged_df.loc[98,"Longitude3_utm_northing"],
                                 only_utm_merged_df.loc[101,"Latitude3_utm_easting"],
                                 only_utm_merged_df.loc[101,"Longitude3_utm_northing"],
                                 only_utm_merged_df.loc[98,"Latitude2_utm_easting"],
                                 only_utm_merged_df.loc[98,"Longitude2_utm_northing"],
                                 only_utm_merged_df.loc[101,"Latitude2_utm_easting"],
                                 only_utm_merged_df.loc[101,"Longitude2_utm_northing"])
angle'''